# Title by Owner

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datetime import datetime
from os import path
from utils.callbacks import SaveBestModelInMemory
from utils.submission import create_submission_zip

from tensorflow.python import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense #, Attention
from utils.attention import Attention
print(5)
# from tensorflow.keras.layers import Attention

5


## Constants

In [3]:
NUM_CLASSES = 12
RANDOM_STATE = 42 # Seed for rng to make everything reproducible and deterministic af
SAVED_MODELS_PATH = "saved-models"
TENSORBOARD_LOGS_PATH = "tensorboard-logs"
SUBMISSIONS_PATH = "../submissions"

## Parameters

In [14]:
BATCH_SIZE = 32 # Number of samples in a mini batch
EPOCHS = 100 # Number of training epochs before the training is stopped
TEST_SPLIT = 0.15 # Percent of data to use for validation/testing

In [15]:
data = np.load(file="../dataset/x_train.npy")
labels = tf.keras.utils.to_categorical(np.load(file="../dataset/y_train.npy"), num_classes=NUM_CLASSES)

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=TEST_SPLIT, random_state=RANDOM_STATE)

# Make sure everything was loaded correctly:
print(f"All samples shape: {data.shape}, all labels shape: {labels.shape}")
print(f"Train samples shape: {train_data.shape}, Train labels shape: {train_labels.shape}")
print(f"Test samples shape: {test_data.shape}, Test labels shape: {test_labels.shape}")

All samples shape: (2429, 36, 6), all labels shape: (2429, 12)
Train samples shape: (2064, 36, 6), Train labels shape: (2064, 12)
Test samples shape: (365, 36, 6), Test labels shape: (365, 12)


## Model Definition

In [9]:
# Create the model inside the function this is a dummy implementation
def build_model(name: str, input_shape: tuple[int,...], classes: int) -> tf.keras.Model:
    # Build the neural network layer by layer
    print(input_shape)

    # X = Input(shape=input_shape)
    # encoder = LSTM(64, return_sequences=True)(X)
    # decoder = Attention()(encoder)
    # Y = Dense(classes)(decoder)

    query_input = tf.keras.Input(shape=(None,), dtype='int32')
    value_input = tf.keras.Input(shape=(None,), dtype='int32')

    token_embedding =  tf.keras.layers.Embedding(input_dim=(36,6), output_dim=128)

    query_embeddings = token_embedding(query_input)
    value_embeddings = token_embedding(value_input)

    cnn_layer = tf.keras.layers.Conv1D(128, 6, padding='same')
    query_seq_encoding = cnn_layer(query_embeddings)
    value_seq_encoding = cnn_layer(value_embeddings)

    query_value_attention_seq = tf.keras.layers.Attention()([query_seq_encoding, value_seq_encoding])

    query_encoding = tf.keras.layers.GlobalAveragePooling1D()(query_seq_encoding)
    query_value_attention = tf.keras.layers.GlobalAveragePooling1D()(query_value_attention_seq)

    X = tf.keras.layers.Concatenate()([query_encoding, query_value_attention])
    Y = Dense(units=classes, activation='softmax')(X)

    # The most promising!!
    # X = Input(shape=input_shape, batch_size=BATCH_SIZE)  # tf.keras.layers.Input(shape=input_shape, name="Input")
    # embedding = Embedding(input_dim=classes, output_dim=128)(X)

    # encoder = LSTM(64, return_sequences=True)(X)
    # encoder = LSTM(units=128, return_sequences=True)(embedding)

    # a = Attention()
    # decoder, attention_weights = Attention(context='many-to-one', alignment_type='global', model_api='functional')(encoder)
    # decoder = Attention(use_scale=False)
    # Y = Dense(units=classes, activation='softmax')(decoder)
    #
    # Connect input and output through the Model class
    model = tf.keras.Model(inputs=X, outputs=Y, name=name)
    # Compile the model
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics="accuracy")

    # Return the model
    return model

In [28]:
# Simple approach
def build_model(name: str, input_shape: tuple[int,...], classes: int) -> tf.keras.Model:
    # Build the neural network layer by layer
    num_samples = 2064
    time_steps = 36

    model_input = Input(shape=(36, 6))
    x = tf.keras.layers.GaussianNoise(0.1)(model_input)
    x = LSTM(128, return_sequences=True)(x)
    # x = LSTM(128)(x)
    x = Attention(64)(x)

    x = tf.keras.layers.Dropout(.5, seed=RANDOM_STATE)(x)

    # Classifier
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    x = tf.keras.layers.Dense(classes, activation="softmax")(x)

    # x = Dense(12)(x)
    model = tf.keras.Model(inputs=model_input, outputs=x, name=name)
    # Compile the model
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics="accuracy")

    # Return the model
    return model


## Training

In [29]:
input_shape = train_data.shape[1:]
print(train_data.shape)
print(input_shape)
classes = NUM_CLASSES
model_name = "My-awesome-model" # Give your model an awesome name for a 2% percent accuracy increase.

model = build_model(model_name, (36,6), classes)
model.summary()

run_id = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
current_tensorboard_log_dir = f"{TENSORBOARD_LOGS_PATH}/{model_name}/{run_id}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=current_tensorboard_log_dir)
print(f"Run tensorboard in a separate process with:\n"
      f"tensorboard --logdir {path.abspath(TENSORBOARD_LOGS_PATH)}\nor\n"
      f"tensorboard --logdir {path.abspath(current_tensorboard_log_dir)}")

best_weights_callback = SaveBestModelInMemory(metric="val_loss")
# reshaped_data = train_data.reshape(2064,256)
model.fit(x=train_data, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(test_data, test_labels), callbacks=[tensorboard_callback, best_weights_callback])

(2064, 36, 6)
(36, 6)
Model: "My-awesome-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 36, 6)]           0         
                                                                 
 gaussian_noise_1 (GaussianN  (None, 36, 6)            0         
 oise)                                                           
                                                                 
 lstm_10 (LSTM)              (None, 36, 128)           69120     
                                                                 
 attention_9 (Attention)     (None, 69)                34048     
                                                                 
 dropout_2 (Dropout)         (None, 69)                0         
                                                                 
 dense_11 (Dense)            (None, 64)                4480      
                            

2022-12-15 13:41:28.910562: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


65/65 [==============================] - 3s 25ms/step - loss: 2.0312 - accuracy: 0.3382 - val_loss: 1.5884 - val_accuracy: 0.4767
Epoch 2/100
 5/65 [=>............................] - ETA: 0s - loss: 1.7105 - accuracy: 0.4125

2022-12-15 13:41:30.769943: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


65/65 [==============================] - 1s 18ms/step - loss: 1.5118 - accuracy: 0.5005 - val_loss: 1.3133 - val_accuracy: 0.5479
Epoch 3/100
65/65 [==============================] - 1s 19ms/step - loss: 1.2851 - accuracy: 0.5644 - val_loss: 1.1632 - val_accuracy: 0.6110
Epoch 4/100
65/65 [==============================] - 1s 19ms/step - loss: 1.1589 - accuracy: 0.6143 - val_loss: 1.1162 - val_accuracy: 0.6329
Epoch 5/100
65/65 [==============================] - 1s 20ms/step - loss: 1.0260 - accuracy: 0.6521 - val_loss: 1.1546 - val_accuracy: 0.6000
Epoch 6/100
65/65 [==============================] - 1s 19ms/step - loss: 0.9742 - accuracy: 0.6642 - val_loss: 1.1098 - val_accuracy: 0.6137
Epoch 7/100
65/65 [==============================] - 1s 19ms/step - loss: 0.8721 - accuracy: 0.7011 - val_loss: 1.1860 - val_accuracy: 0.6082
Epoch 8/100
65/65 [==============================] - 1s 19ms/step - loss: 0.8307 - accuracy: 0.7083 - val_loss: 1.1221 - val_accuracy: 0.6493
Epoch 9/100
65/65 

KeyboardInterrupt: 

## Optional: Save model in memory

In [29]:
model.set_weights(best_weights_callback.best_weights)
saved_model_path = f"{SAVED_MODELS_PATH}/{model_name}/{run_id}"
model.save(saved_model_path)

INFO:tensorflow:Assets written to: saved-models/My-awesome-model/2022-12-12-19-48-03/assets


INFO:tensorflow:Assets written to: saved-models/My-awesome-model/2022-12-12-19-48-03/assets


## Optional: Create submission ZIP

In [30]:
submission_path = f"{SUBMISSIONS_PATH}/{model_name}/{run_id}"
create_submission_zip(submission_path, saved_model_path)

print(f"Created submission: {submission_path}.zip")

Created submission: ../submissions/My-awesome-model/2022-12-12-19-48-03.zip
